# Import libraries

In [8]:
import pandas as pd
import numpy as np
import pickle
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing,metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from nltk.corpus import stopwords
nltk.download('punkt')
st = ISRIStemmer()
nltk.download('stopwords')
stop=stopwords.words('arabic')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\XPS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\XPS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Preprocessing


In [ ]:
from jpype import startJVM, shutdownJVM, java
import os
import sys
from pathlib import Path
import jpype
from jpype import *

def extraire_colonne_secondaire(texte):
    mots = texte.split('{{')
    colonne_secondaire = [mot.split(':')[1][:-2] if '}}' in mot else '' for mot in mots]
    colonne_secondaire = [mot.strip('}').strip() for mot in colonne_secondaire if mot]
    return colonne_secondaire


def steem(text):
    #print(jpype.getDefaultJVMPath())
    #path1='c:\\tmp'
    
    #if not jpype.isJVMStarted():
        #jpype.startJVM(jpype.getDefaultJVMPath() ,'-ea', classpath=[path1],convertStrings=True )
    
  
    
    path3= 'C:/Users/XPS/Desktop/FerhatOumaimaQassimiIkhlas/ADAT-Analyzer.jarADAT-Analyzer.jar'
    jpype.addClassPath(path3)
    
    #Import the Java class
    LemmaStemma = jpype.JClass("net.oujda_nlp_team.ADATAnalyzer")
    #POStag = jpype.JClass("net.nlp.parser.TestParserClient")
    
    # Chemin vers le fichier
    #chemin_fichier = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\niveaux\1\استئجار شقة.txt"
    #chemin_fichier = r"C:\Users\mimif\OneDrive\Bureau\S3\maz\Ressources lisibilité\niveaux\test.txt"
    
    #with open(chemin_fichier, encoding="utf8") as file:
        #file_content = file.read()
        
    #Lemmatizer, type de retour String
    #lemmes = LemmaStemma.getInstance().processADATLemmatizerString(file_content)
    
    #Stemmer, type de retour String
    stems = LemmaStemma.getInstance().processADATStemmerString(text)

    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    #lemmes = extraire_colonne_secondaire(lemmes)
    stems = extraire_colonne_secondaire(stems)

     # Retirer les marqueurs "{{" et "}}" de chaque élément avant de les fusionner
    #lemmes = [lemme.replace('{{', '').replace('}}', '') for lemme in lemmes]
    stems = [stem.replace('{{', '').replace('}}', '') for stem in stems]
    # Concaténer les lemmes et les stems en une seule chaîne de caractères
    #lemme_string = ' '.join(lemmes)
    stem_string = ' '.join(stems)
    return stem_string
     #return  lemme_string, stem_string



In [9]:

def clean(text):
  #remove all English chars 
  text = re.sub(r'\s*[A-Za-z]\s*', ' ' , text)
  #remove hashtags
  text = re.sub("#", " ", text)
  #remove all numbers 
  text = re.sub(r'\[0-9]*\]',' ',text)
  #remove duplicated chars
  text = re.sub(r'(.)\1+', r'\1', text)
  #remove :) or :(
  text = text.replace(':)', "")
  text = text.replace(':(', "")
  #remove multiple exclamation
  text = re.sub(r"(\!)\1+", ' ', text)
  #remove multiple question marks
  text = re.sub(r"(\?)\1+", ' ', text)
  #remove multistop
  text = re.sub(r"(\.)\1+", ' ', text)
  #remove additional spaces
  text = re.sub(r"[\s]+", " ", text)
  text = re.sub(r"[\n]+", " ", text)
  
  return text

def remStopWords(Text):
  return " ".join(word for word in Text.split() if word not in stop)

def stemWords(Text):
  return " ".join(st.stem(word) for word in Text.split())

In [10]:
#read data

dataFrame = pd.read_csv('res.csv')

dataFrame['Contenu']=dataFrame['Contenu'].apply(lambda Text: remStopWords(str(Text)))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text : clean(Text))
dataFrame['Contenu']=dataFrame['Contenu'].apply(lambda Text: stemWords(Text))

dataFrame.drop_duplicates(subset ="Contenu",keep = 'first', inplace = True)

train_x, valid_x, train_y, valid_y = train_test_split(dataFrame['Contenu'], dataFrame['classe'],test_size=0.2,random_state=42)

before_encode_valid_y = dataFrame['classe'].unique()

encoder = preprocessing.LabelEncoder()

train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=5000)

tfidf_vect.fit(dataFrame['Contenu'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)

# Model Training

In [4]:
def train_model(classifier, feature_vector_train, label,feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    return metrics.accuracy_score(predictions, valid_y),predictions

In [5]:
params_grid = [
                    {'kernel': ['linear']}]

clf_svm = GridSearchCV(SVC(), params_grid, cv=5)

In [6]:
from sklearn.metrics import confusion_matrix, classification_report

# Entraîner le modèle SVM
clf_svm.fit(xtrain_tfidf, train_y)

# Prédictions sur les données de validation
predictions = clf_svm.predict(xvalid_tfidf)

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(valid_y, predictions)


# Calculer et afficher d'autres indicateurs de performance
print("\nRapport de classification :")

    # Générer le rapport de classification
report = classification_report(valid_y, predictions, zero_division=1)

    # Extraire uniquement la partie du rapport de classification contenant le tableau
report_lines = report.split('\n')[:-3]  # Exclure les lignes d'en-tête et de pied de page

    # Imprimer le tableau de classification
print(f"Tableau de classification pour le noyau '{kernel}':")
print('\n'.join(report_lines))  # Imprimer les lignes du tableau de classification

print("\n")


Rapport de classification :


NameError: name 'kernel' is not defined

In [17]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

'''#read data
dataFrame = pd.read_csv('res.csv')

dataFrame['Contenu']=dataFrame['Contenu'].apply(lambda Text: remStopWords(str(Text)))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text : clean(Text))
dataFrame['Contenu']=dataFrame['Contenu'].apply(lambda Text: stemWords(Text))

dataFrame.drop_duplicates(subset ="Contenu",keep = 'first', inplace = True)

train_x, valid_x, train_y, valid_y = train_test_split(dataFrame['Contenu'], dataFrame['classe'],test_size=0.2,random_state=(50))

before_encode_valid_y = dataFrame['classe'].unique()

encoder = preprocessing.LabelEncoder()

train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

tfidf_vect = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}', max_features=50000)

tfidf_vect.fit(dataFrame['Contenu'])

xtrain_tfidf = tfidf_vect.transform(train_x)
xvalid_tfidf = tfidf_vect.transform(valid_x)'''
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack# Importez vos fonctions personnalisées

# Lecture des données à partir du fichier CSV
dataFrame = pd.read_csv('res.csv')

# Prétraitement du contenu
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: remStopWords(str(Text)))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: clean(Text))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: stemWords(Text))

# Suppression des doublons
dataFrame.drop_duplicates(subset="Contenu", keep='first', inplace=True)

# Division en ensembles d'entraînement et de validation
train_data, valid_data = train_test_split(dataFrame, test_size=0.2, random_state=42)

# Encodage des étiquettes de classe
encoder = LabelEncoder()
train_data['classe'] = encoder.fit_transform(train_data['classe'])
valid_data['classe'] = encoder.transform(valid_data['classe'])

# Extraction des caractéristiques TF-IDF
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=50000)
tfidf_vect.fit(dataFrame['Contenu'])

xtrain_tfidf = tfidf_vect.transform(train_data['Contenu'])
xvalid_tfidf = tfidf_vect.transform(valid_data['Contenu'])

# Concaténation des nouvelles caractéristiques avec les caractéristiques TF-IDF
nouvelles_caracteristiques_train = train_data[['SL1', 'WL1', 'WL4', 'WL5', 'AMb1', 'VL1', 'VL2']].values
nouvelles_caracteristiques_valid = valid_data[['SL1', 'WL1', 'WL4', 'WL5', 'AMb1', 'VL1', 'VL2']].values
xtrain_combined = hstack((xtrain_tfidf, nouvelles_caracteristiques_train))
#print(xtrain_combined)
xvalid_combined = hstack((xvalid_tfidf, nouvelles_caracteristiques_valid))



# Liste des noyaux SVM à tester
kernels = ['linear', 'rbf', 'poly', 'sigmoid']
for kernel in kernels:
    # Initialiser le modèle SVM avec le noyau actuel
    svm_model = SVC(kernel=kernel)

    # Entraîner le modèle SVM
    svm_model.fit(xtrain_combined, train_data['classe'])

    # Faire des prédictions sur l'ensemble de test
    predictions = svm_model.predict(xvalid_combined)

    # Générer le rapport de classification
    report = classification_report(valid_data['classe'], predictions, zero_division=1)

    # Extraire uniquement la partie du rapport de classification contenant le tableau
    report_lines = report.split('\n')[:-3]  # Exclure les lignes d'en-tête et de pied de page

    # Imprimer le tableau de classification
    print(f"Tableau de classification pour le noyau '{kernel}':")
    print('\n'.join(report_lines))  # Imprimer les lignes du tableau de classification

    print("\n")

Tableau de classification pour le noyau 'linear':
              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.50      1.00      0.67         4
           2       0.84      0.80      0.82        20
           3       0.67      0.62      0.64        13
           4       0.50      0.56      0.53         9

    accuracy                           0.70        54


Tableau de classification pour le noyau 'rbf':
              precision    recall  f1-score   support

           0       0.50      0.50      0.50         8
           1       1.00      0.00      0.00         4
           2       0.45      0.95      0.61        20
           3       1.00      0.00      0.00        13
           4       0.75      0.33      0.46         9

    accuracy                           0.48        54


Tableau de classification pour le noyau 'poly':
              precision    recall  f1-score   support

           0       0.50      0.88   

Save Model

In [18]:
from joblib import dump, load
import joblib

# Choix du noyau SVM avec la meilleure précision
best_kernel = 'linear'  #  'linear' a la meilleure précision, vous devez remplacer cela par le noyau avec la meilleure précision
joblib.dump(tfidf_vect, 'tfidf_vect.joblib')

# Save the label encoder
joblib.dump(encoder, 'label_encoder.joblib')
# Initialisation du modèle SVM avec le meilleur noyau
best_svm_model = SVC(kernel='linear')

# Entraînement du modèle SVM avec les données d'entraînement combinées
best_svm_model.fit(xtrain_combined, train_data['classe'])

dump(best_svm_model, 'modele_svm.joblib')




['modele_svm.joblib']

In [14]:
import pickle
kernel= 'linear'
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack# Importez vos fonctions personnalisées
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Lecture des données à partir du fichier CSV
dataFrame = pd.read_csv('res.csv')

# Prétraitement du contenu
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: remStopWords(str(Text)))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: clean(Text))
dataFrame['Contenu'] = dataFrame['Contenu'].apply(lambda Text: stemWords(Text))

# Suppression des doublons
dataFrame.drop_duplicates(subset="Contenu", keep='first', inplace=True)

# Division en ensembles d'entraînement et de validation
train_data, valid_data = train_test_split(dataFrame, test_size=0.2, random_state=42)

# Encodage des étiquettes de classe
encoder = LabelEncoder()
train_data['classe'] = encoder.fit_transform(train_data['classe'])
valid_data['classe'] = encoder.transform(valid_data['classe'])

# Extraction des caractéristiques TF-IDF
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}')
tfidf_vect.fit(dataFrame['Contenu'])

xtrain_tfidf = tfidf_vect.transform(train_data['Contenu'])
xvalid_tfidf = tfidf_vect.transform(valid_data['Contenu'])

# Concaténation des nouvelles caractéristiques avec les caractéristiques TF-IDF
nouvelles_caracteristiques_train = train_data[['SL1', 'WL1', 'WL4', 'WL5', 'AMb1', 'VL1', 'VL2']].values
nouvelles_caracteristiques_valid = valid_data[['SL1', 'WL1', 'WL4', 'WL5', 'AMb1', 'VL1', 'VL2']].values
xtrain_combined = hstack((xtrain_tfidf, nouvelles_caracteristiques_train))
#print(xtrain_combined)
xvalid_combined = hstack((xvalid_tfidf, nouvelles_caracteristiques_valid))



    # Initialiser le modèle SVM avec le noyau actuel
svm_model = SVC(kernel='linear')

    # Entraîner le modèle SVM
svm_model.fit(xtrain_combined, train_data['classe'])

    # Faire des prédictions sur l'ensemble de test
predictions = svm_model.predict(xvalid_combined)
print(predictions)
    # Générer le rapport de classification
report = classification_report(valid_data['classe'], predictions, zero_division=1)

    # Extraire uniquement la partie du rapport de classification contenant le tableau
report_lines = report.split('\n')[:-3]  # Exclure les lignes d'en-tête et de pied de page

    # Imprimer le tableau de classification
print(f"Tableau de classification pour le noyau :")
print('\n'.join(report_lines))  # Imprimer les lignes du tableau de classification

print("\n")
# Enregistrement du modèle SVM avec le noyau linéaire
with open('modele_svm_linear.pkl', 'wb') as f:
    pickle.dump(svm_model, f)


[1 2 2 2 3 2 3 1 2 2 4 4 4 4 4 4 4 2 2 1 4 4 1 1 3 1 0 3 2 2 2 2 1 3 3 0 2
 2 2 2 0 1 3 0 2 3 3 3 2 3 0 2 4 3]
Tableau de classification pour le noyau :
              precision    recall  f1-score   support

           0       1.00      0.62      0.77         8
           1       0.50      1.00      0.67         4
           2       0.84      0.80      0.82        20
           3       0.67      0.62      0.64        13
           4       0.50      0.56      0.53         9

    accuracy                           0.70        54




In [23]:
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack# Importez vos fonctions personnalisées
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
# Charger le modèle SVM sauvegardé
svm_model = joblib.load('modele_svm.joblib')

# Charger les données à partir du fichier CSV
dataFrame_new = pd.read_csv('fichier.csv')

# Prétraitement du contenu des nouvelles données
dataFrame_new['Contenu'] = dataFrame_new['Contenu'].apply(lambda Text: remStopWords(str(Text)))
dataFrame_new['Contenu'] = dataFrame_new['Contenu'].apply(lambda Text: clean(Text))
dataFrame_new['Contenu'] = dataFrame_new['Contenu'].apply(lambda Text: stemWords(Text))
# Load the TF-IDF vectorizer
tfidf_vect = joblib.load('tfidf_vect.joblib')

# Transform the new data with the TF-IDF vectorizer
xvalid_tfidf_new = tfidf_vect.transform(dataFrame_new['Contenu'] )

# Extraction des caractéristiques TF-IDF pour les nouvelles données
#tfidf_vect = TfidfVectorizer()
#xvalid_tfidf_new = tfidf_vect.transform(dataFrame_new['Contenu'])

# Concaténation des nouvelles caractéristiques avec les caractéristiques TF-IDF
nouvelles_caracteristiques_new = dataFrame_new[['SL1', 'WL1', 'WL4', 'WL5', 'AMb1', 'VL1', 'VL2']].values
xvalid_combined_new = hstack((xvalid_tfidf_new, nouvelles_caracteristiques_new))

# Faire des prédictions sur les nouvelles données
predictions_new = svm_model.predict(xvalid_combined_new)

# Afficher les prédictions
print(predictions_new)


[1]
